## Overview

Welcome to the first notebook of the fba workshops! The learning objectives of this notebook are as follows:

- Understanding flux-balance analysis
- Understanding the use of GEMs/GSMMs in flux-balance analysis
- Getting familiar with the COBRApy framework
    - Importing a GSMM
    - Finding reactions & metabolites
    - Objective functions and running the analysis
    - Altering reaction upper and lower bounds

The take-home practice section will then try to give you intuition on:

- hybrid objective functions
- competing pathways
- knock-outs

**Before getting started:** Please put the names of your group members into the markdown cell below so that you can take credit for your work.

**Group Members:**

*Alexis Moutafis-Tymcio*

## What is flux-balance analysis?

Flux-balance analysis is a predictive modeling method that is significantly used in systems biology. It has a wide variety of applications including (but not limited to):

1. Media optimization for the culture
2. Finding knock-out targets
3. Understanding metabolic constraints
4. Neuroscience
5. Biomanufacturing
6. Cancer Research

Many iGEM teams have cited [this paper](https://www.nature.com/articles/nbt.1614) to explain how flux-balance analysis works. In a nutshell, it derives from the idea that reactions and metabolites in a metabolism can be represented as a matrix by the following **example:** 

Let's take an arbitrary reaction:

$$2Na + 2HCl \rightarrow 2NaCl + H_2$$

This reaction can be represented as a column vector:

$$v = \begin{pmatrix} -2 & -2 & 2 & 1 \end{pmatrix}^T$$

Where each item represents the coefficient consumed/produced of that metabolite in the reaction. This principle applied to thousands of metabolites and reactions, and we get a linear optimization problem formulated as $Sv = 0$ where:

- $S \in R^{M \times N}$ represents the stoichiometric matrix with each column a reaction, and each row a metabolite.
- $v \in R^{N}$ represents the fluxes (rates) of reactions, usually in units $mmol/gDW/h$ (biomass reactions are usually in $1/h$). 
- an assumption is made that the metabolic network is at steady-state, where each metabolite is consumed and produced at the same amount.

Given a metabolite concentration $[x]$ with the corresponding coefficient $s_i$ in the $i^{th}$ reaction, and $t$ for time, an equivalent formulation of the problem would be $$ \sum_{i=1}^N s_i v_i = \frac{d[x]}{dt} = 0 $$

For more advanced analyses, flux-balance analysis pairs with [multiomics data](https://en.wikipedia.org/wiki/Multiomics) which can introduce more information such as genetics, transcriptomics, epigenetics, etc.

### What are genome-scale metabolic models?

Genome-scale metabolic models (GSMM or GEM for short) are formatted data files that contains all the essential information of the metabolism 
of an organism. Not only the set of reactions and metabolites, but they hold much more information such as gene associations (for turning on/off phenotypes), model compartments, flux constraints, etc. Development of comprehensive and accurate GSMMs for non-model organisms is one of the greatest bottlenecks of systems biology.

Most GSMMs are published and used in the file format called SBML, which stands for **systems biology markup language**. However, the JSON file format, originally created for data transfer in web software applications, is also used for a similar purpose.

### The COBRA Toolbox

The [iJO1366](http://bigg.ucsd.edu/models/iJO1366) model was developed to mimic the metabolism of *Escherichia coli*. In the following practice, you'll be introduced to the basics of the python library COBRApy that is used to compile these models and run very basic forms of flux-balance analysis.

First start off by installing COBRApy and making necessary imports into the notebook.

In [2]:
%pip install cobra

from cobra import io, Model, Solution
import argparse, os, sys

Note: you may need to restart the kernel to use updated packages.


Now, we read our model file into a `Cobra.Model` object.

In [5]:
model: Model = io.read_sbml_model('./iJO1366.xml')

five_mets = [f'\n({i+1}) ID: {met.id} - Name: {met.name}' for i, met in enumerate(model.metabolites[:5])]
five_rxns = [f'\n({i+1}) ID: {rxn.id} - Name: {rxn.name}' for i, rxn in enumerate(model.reactions[:5])]
print(f"First 5 metabolites: {''.join(five_mets)}\n")
print(f"First 5 reactions: {''.join(five_rxns)}")

First 5 metabolites: 
(1) ID: 10fthf_c - Name: 10-Formyltetrahydrofolate
(2) ID: 12dgr120_c - Name: 1,2-Diacyl-sn-glycerol (didodecanoyl, n-C12:0)
(3) ID: 12dgr140_c - Name: 1,2-Diacyl-sn-glycerol (ditetradecanoyl, n-C14:0)
(4) ID: 12dgr141_c - Name: 1,2-Diacyl-sn-glycerol (ditetradec-7-enoyl, n-C14:1)
(5) ID: 12dgr160_c - Name: 1,2-Diacyl-sn-glycerol (dihexadecanoyl, n-C16:0)

First 5 reactions: 
(1) ID: EX_cm_e - Name: Chloramphenicol exchange
(2) ID: EX_cmp_e - Name: CMP exchange
(3) ID: EX_co2_e - Name: CO2 exchange
(4) ID: EX_cobalt2_e - Name: Co2+ exchange
(5) ID: DM_4crsol_c - Name: Sink needed to allow p-Cresol to leave system


The above code reads the `sbml` file into the `Model` object, and then reads out the names and ids of the first 5 reactions and metabolites in the array. To understand how data is stored in the `Model` object, we can further look into the `Metabolite` and `Reaction` classes, some of which we'll cover in this notebook. If you're curious, you can visit the [COBRApy documentation page](https://cobrapy.readthedocs.io/en/latest/) for more information.

Here's a neater way to display the model information on jupyter notebook, which neatly renders as a table:

In [6]:
model

Name,iJO1366
Memory address,25d37619610
Number of metabolites,1805
Number of reactions,2583
Number of genes,1367
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"cytosol, extracellular space, periplasm"


As previously discussed, flux-balance analysis makes a major assumption; that all metabolites are produced and consumed at the same amount. This assumption describes a steady-state. Within this assumption, we're trying to find an arrangement of flux vector $v$ that gives the highest (maximum) flux through our target reaction (objective).

Let's assume we want to optimize for the **biomass** objective function. The name of this function may vary based on the GSMM and the database that was used to construct it, but the **biomass** keyword is usually in there somewhere.

With the following script, we can identify the biomass reaction by looking for the **biomass** keyword in all reaction IDs.

In [7]:
biomass_rxns = [rxn for rxn in model.reactions if 'biomass' in rxn.id.lower()]

print(f'Biomass Reactions: ')
for rxn in biomass_rxns:
    print(f'- {rxn.id}: {rxn.name}')

Biomass Reactions: 
- BIOMASS_Ec_iJO1366_WT_53p95M: E. coli biomass objective function (iJO1366) - WT - with 53.95 GAM estimate
- BIOMASS_Ec_iJO1366_core_53p95M: E. coli biomass objective function (iJO1366) - core - with 53.95 GAM estimate


Some models may have multiple biomass functions. Usually different biomass functions indicate 

1. the estimations via different experiments, or 
2. predictions under different medium conditions, representing different "modes" of the organism.

As an example, in our [iGEM 2025 (Sterosaurus)](https://2025.igem.wiki/mcmasteru/) project, we used the iCre1355 model of the *C. reinhardtii* algae organism. There were 3 biomass functions representing the autotrophic, mixotrophic, and heterotrophic mode. These modes represent the growth function of the organism under different medium conditions i.e. O2 stress, CO2 stress, etc. You can find the paper for the GSMM model we used in our project [here](https://onlinelibrary.wiley.com/doi/10.1111/tpj.13059).

For our optimization purposes, we'll use the core biomass function `BIOMASS_Ec_iJO1366_core_53p95M`. Let's now optimize our model and get the flux results.

In [8]:
biomass_obj = 'BIOMASS_Ec_iJO1366_core_53p95M'

model.objective = biomass_obj
solution: Solution = model.optimize()

solution.fluxes.sort_values(ascending=False, inplace=True)
print(f'Top 5 fluxes in optimal solution:')
print(solution.fluxes.head(5))

print(f'\nOptimal growth rate: {solution.objective_value} 1/hr')

Top 5 fluxes in optimal solution:
ATPS4rpp      55.815247
EX_h2o_e      45.619430
CYTBO3_4pp    35.149539
NADH16pp      31.021723
EX_co2_e      19.675223
Name: fluxes, dtype: float64

Optimal growth rate: 0.9823718127269743 1/hr


Now, moving on to different features.

Additionally, reactions can be upper and lower bounded within the model. We can do this by reaching a reaction of our choice within the model and change its `upper_bound` and `lower_bound` parameters. Let's try to do this on the medium uptake of our model.

To find a reaction of interest, we may think we need to iterate through every reaction in the model. However, there's simple ways in which we can reduce the workload. For instance, the cobra `Model` object is known to keep record of the compartments of each metabolite. We can first write a script to print out the compartments available in the model, and then filter only for the exchange reactions that have extracellular metabolites as reactants.

We start by listing out the compartments of our model.

In [9]:
model.compartments

{'c': 'cytosol', 'e': 'extracellular space', 'p': 'periplasm'}

Notice that the `'e': 'extracellular space'` compartment fits our use case. We're looking for the extracellular oxygen metabolite to find its relevant reactions.

In [10]:
# Query for all metabolites with 'o2' in their ID or name
mets = model.metabolites.query('.*o2.*')
# Print only those in the extracellular compartment
output = []
for met in mets:
    if met.compartment != 'e':
        continue
    output.append((met.id, met.name))

output

[('co2_e', 'CO2 CO2'),
 ('h2o2_e', 'Hydrogen peroxide'),
 ('kdo2lipid4_e', 'KDO(2)-lipid IV(A)'),
 ('so2_e', 'Sulfur dioxide'),
 ('no2_e', 'Nitrite'),
 ('o2_e', 'O2 O2'),
 ('o2s_e', 'Superoxide anion')]

The sixth item seems to be the metabolite we're loooking for. Now, let's try to constrain the model based on its oxygen uptake. We know that the oxygen uptake reaction must contain our metabolite as a reactant. So let's find the oxygen uptake reaction.

In [11]:
o2e_id, o2e_name = output[5]

rxns = []
for rxn in model.reactions:
    if o2e_id in [met.id for met, coef in rxn.metabolites.items() if coef < 0]:
        rxns.append(rxn)

for r in rxns:
    print(f'- {r.id}: {r.name}: {r.lower_bound} to {r.upper_bound} mmol/gDW/hr')

- EX_o2_e: O2 exchange: -1000.0 to 1000.0 mmol/gDW/hr
- O2tex: Oxygen transport via diffusion (extracellular to periplasm): -1000.0 to 1000.0 mmol/gDW/hr


The above code filters the reactions that include our metabolite as a reactant, and prints out their id, name, and constraints. In the context of a metabolic model, a range of -1000 to 1000 as upper and lower bounds is practically infinite. Let's set this much lower. The reason why we take the copy of the original model in the below code is because `Model` and `Reaction` behave as reference types, and the variable `oxygen_uptake_rxn` can mutate the original object in the model. Making a copy prevents us from overwriting the original data.

In the below cell block, we use the `upper_bound` and `lower_bound` properties of the `Reaction` object to constrain its activity. Upon the update, we run the analysis using the `optimize()` method of our model.

In [12]:
# Original optimization
solution = model.optimize()

print(f'\nOptimal growth rate: {solution.objective_value} 1/hr')

# With new boundaries
copy_model = model.copy()
oxygen_uptake_rxn = copy_model.reactions.get_by_id('EX_o2_e')
oxygen_uptake_rxn.lower_bound = -5.0
oxygen_uptake_rxn.upper_bound = 5.0 

solution = copy_model.optimize()

print(f'\nOptimal growth rate: {solution.objective_value} 1/hr')


Optimal growth rate: 0.9823718127269705 1/hr

Optimal growth rate: 0.49154514798795895 1/hr


Now that we've covered some of the basics of COBRApy, we can get started with some take-home practice questions.

## Take-Home Practice

#### Part 1: Hybrid Objective for Optimization

In the above section, recall that we could set the objective function of a model by calling `model.objective = 'reaction_id'`. However, there's an alternative way in which we can define the model objective. If you read the documentation for the `Reaction` object, you'll see that one of its many properties is called `objective_coefficient` which is a float integer. This way, we can include multiple reactions in the objective function. It is important to note that all of the objective coefficients should add up to 1.

A demonstration:

In [13]:
model = io.read_sbml_model('./iJO1366.xml')

# Reset objective coefficients
for rxn in model.reactions:
    rxn.objective_coefficient = 0.0

# Set new objective
model.reactions.get_by_id('EX_glc__D_e').objective_coefficient = 0.5
model.reactions.get_by_id('EX_o2_e').objective_coefficient = 0.5


model.optimize()
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,0.2864,6,100.00%
o2_e,EX_o2_e,0.5727,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,-0.5727,2,66.67%
co2_e,EX_co2_e,-0.5727,1,33.33%
h2o_e,EX_h2o_e,-0.5727,0,0.00%
h_e,EX_h_e,-0.5727,0,0.00%


In the cell below, your task is to:

<input type="checkbox" style="margin-bottom: 20px;"><label>Import the model for *Saccharomyces cerevisiae* (file named `yeast-GEM.xml`) into COBRA</label></input><br>
<input type="checkbox" style="margin-bottom: 20px;"><label>Set a custom objective function where: (1) thiazole phosphate synthesis (`"id: r_2070"`) objective coefficient is set to 0.5 and (2) biomass function (`"id: r_4041"`) coefficient set to 0.5</label></input><br>
<input type="checkbox"><label>Optimize the model and print out the resulting fluxes for the two reactions. (<b>Hint:</b> Use `solution.fluxes[['rid1','rid2']]` to print results)</label></input><br>

In [ ]:
model2: Model = io.read_sbml_model('./yeast-GEM.xml')

# Reset objective coefficients
for rxn in model2.reactions:
    rxn.objective_coefficient = 0.0

# Set new objective
model2.reactions.get_by_id('r_2070').objective_coefficient = 0.5
model2.reactions.get_by_id('r_4041').objective_coefficient = 0.5

solution = model2.optimize()
print(solution.fluxes[['r_2070', 'r_4041']])

r_2070    1.004973e-07
r_4041    8.374771e-02
Name: fluxes, dtype: float64


#### Part 2: Gene Knock-out

We can simulate the response of the metabolism upon knocking out different genes. We can do this with the `Gene` object in the library. We can access the gene information and knock it out by accessing the `gene_reaction_rule` property of the `Reaction` object, which returns an identifier string. We can then query that string using `model.genes.get_by_id('my_gene_rule_id')`, save it in a variable called `gene` and simply run `gene.knock_out()` to knock it out.

<br>

To carry out this step, you'll need to perform the following steps on the same yeast metabolism from part 1:

<input type="checkbox" style="margin-bottom: 20px;"><label>Re-set the objective coefficients of the model (or simply import it again)</label></input><br>
<input type="checkbox" style="margin-bottom: 20px;"><label>Make 2 copies of the model (for 2 different knock-out experiments)</label></input><br>
<input type="checkbox" style="margin-bottom: 20px;"><label>On the first copy, knock out the pyruvate kinase reaction from its gene. The reaction has `id: r_0962`.</label></input><br>
<input type="checkbox" style="margin-bottom: 20px;"><label>On the second copy, knock out the acetyl-CoA carboxylase from its gene. The reaction has `id: r_0109`.</label></input><br>
<input type="checkbox" style="margin-bottom: 20px;"><label>Set the objective function to `id: r_4041` (for biomass) on the original model, and the two copies (3 models in total).</label></input><br>
<input type="checkbox"><label>Run flux-balance analysis on each of the 3 models and print out the flux of the objective reaction for clear comparison.</label></input><br>

<br>

**Please note that a reaction can be regulated by multiple genes.**

<br>

Upon completion, answer the follow-up questions:

1. Are the flux values of `r_4041` the same between the original model and the first copy? What does this difference tell us about the relation of the knocked out gene to the biomass objective?
2. Are the flux values of `r_4041` the same between the original model and  the second copy? What does this difference tell us about the relation of the knocked out gene to the biomass objective?
3. If you mess around in COBRApy, you may notice that we can also perform knock-out on the reaction itself by setting its upper and lower bounds to 0 or simply running `reaction.knock_out()`. This seems as an easier approach to perform a knock-out. Do you think we should use this method for knock-outs instead of the gene rule? Can you explain why (or why not) this is a good (or bad) approach?

In [29]:
# Reset objective coefficients
for rxn in model2.reactions:
    rxn.objective_coefficient = 0.0

model_pk_ko = model2.copy()
model_acc_ko = model2.copy()

#Pyruvate kinase KO
rxn_pk = model_pk_ko.reactions.get_by_id('r_0962')
rule_pk = rxn_pk.gene_reaction_rule
print(rule_pk)

for gene in rxn_pk.genes:
    gene.knock_out()

#Acetyl-CoA KO
rxn_acc = model_acc_ko.reactions.get_by_id('r_0109')
rule_acc = rxn_acc.gene_reaction_rule
print(rule_acc)

for gene in rxn_acc.genes:
    gene.knock_out()

#Objective functions
model2.reactions.get_by_id('r_4041').objective_coefficient = 0.5
model_pk_ko.reactions.get_by_id('r_4041').objective_coefficient = 0.5
model_acc_ko.reactions.get_by_id('r_4041').objective_coefficient = 0.5

solution = model2.optimize()
solution_pk = model_pk_ko.optimize()
solution_acc = model_acc_ko.optimize()

print("\n")

print(solution.fluxes[['r_4041']])
print(solution_pk.fluxes[['r_4041']])
print(solution_acc.fluxes[['r_4041']])


YAL038W or YOR347C
YDL141W and YNR016C


r_4041    0.083748
Name: fluxes, dtype: float64
r_4041    0.07982
Name: fluxes, dtype: float64
r_4041    1.285378e-18
Name: fluxes, dtype: float64


**Answer to 1:** There was a relatively minimal decrease in flux when we knocked out pyruvate kinase. This would imply that it is not required for biomass production, or other pathways exist.

**Answer to 2:** There was a very stark decrease (basically 0) in flux when we knocked out acetyl-CoA. This implies that this compound is critical for biomass production.

**Answer to 3:** I don't think that's a good approach to take. By knocking out the entire reaction, and not a specific gene, you over-generalize (i.e. for an "or" reaction, knocking out one gene could still let the reaction happen). This is too broad of an assumption and could create inaccurate results.

In this notebook, we've explored what flux-balance analysis is, and how we can use COBRApy to load GSMMs from `sbml` files and optimize for a certain objective function.

In most applications of flux-balance analysis, it is valuable to read the paper of the GSMMs model release as it contains important information relevant to our purpose.

In the upcoming activities, we'll expand on the practice we did here and introduce new concepts such as flux-variability analysis, knock-out targets, troubleshooting and validating models.

### Supplementary Material

As previously mentioned, feel free to check out the [COBRApy documentation](https://cobrapy.readthedocs.io/en/latest/) on the use of the library.

If you're curious, you can also check out [our software repository](https://gitlab.igem.org/2025/software-tools/mcmasteru) from iGEM 2025, where the flux-balance analysis code can be found under the `fba-codebase` folder. Alternatively, you can also find the code on this [GitHub repo](https://github.com/FarukEfe/McMasterU-iGEM-2025-Modelling).

If you're looking for more resources Dr. Francisco Zorrilla from Cambridge University developed a set of exercises for his Part III Systems Biology course. You can check out his resources [here](https://github.com/franciscozorrilla/systems-biology-fba-practical).